In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [13]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [14]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: arbitraryyy, just to make sure after filtered, it's not empty
dummy_data = np.random.uniform(1, 100, len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

# where(element > 30)
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
gt30_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 30].reshape(1,-1,1)
dummy_theory_output = torch.exp(torch.mean(torch.log(gt30_dummy_data_tensor)))



In [15]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # where part
        num_fil_X = torch.sum((X>30).double())
        fil_X = torch.tensor([ele[0] if ele[0]>30 else 1 for ele in X[0]])
        # fil_X = torch.where(X>30, X, 1)
        return (torch.abs((torch.log(self.w)*num_fil_X)-torch.sum(torch.log(fil_X)))<=num_fil_X*torch.log(torch.tensor(1.01)), self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_88041/1017795205.py:11: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  fil_X = torch.tensor([ele[0] if ele[0]>30 else 1 for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_88041/1017795205.py:11: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  fil_X = torch.tensor([ele[0] if ele[0]>30 else 1 for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_88041/1017795205.py:11: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use 

In [16]:
# prover calculates settings, send to verifier
gt30_data_tensor = data_tensor[data_tensor > 30].reshape(1,-1,1)
print("new tensor: ", gt30_data_tensor.size())
theory_output = torch.exp(torch.mean(torch.log(gt30_data_tensor)))
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        
    def forward(self,X):
        # where part
        num_fil_X = torch.sum((X>30).double())
        fil_X = torch.tensor([ele[0] if ele[0]>30 else 1 for ele in X[0]])
        # fil_X = torch.where(X>30, X, 1)

        return (torch.abs((torch.log(self.w)*num_fil_X)-torch.sum(torch.log(fil_X)))<=num_fil_X*torch.log(torch.tensor(1.01)), self.w)

    

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path,[8], "resources", settings_path)

new tensor:  torch.Size([1, 272, 1])
Theory_output:  tensor(50.8632)
==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-196532,29696],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":1514,"total_const_size":5,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.49927}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_88041/609443682.py:15: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  fil_X = torch.tensor([ele[0] if ele[0]>30 else 1 for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_88041/609443682.py:15: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  fil_X = torch.tensor([ele[0] if ele[0]>30 else 1 for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_88041/609443682.py:15: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use thi

In [17]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 25.365365266799927 seconds
Theory output:  tensor(50.8632)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 50.86328125
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[11768814371718170976, 435173728250646979, 519717007263840094, 1741290966923863957], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [9740814119635710701, 8723924064432029923, 17927155970413989335, 152971583043225146]]], 'proof': '2c11f90c783f3ffabcfecf8c573d5ac2b434f642ff4d57a1d081323a194bc5eb1f29c6fff5a4b7a9dbdb79c78ec896b4210aa486f7b975dc4b6bf7ccf443173c25c443bd2018c6c5a4001610e67bcf57824c04ecc8bb38919ef2e446a7cd67772787e44fea30ab6e39bf075954fd6c57c6f32e85b614f99e52d457e65cf7cdf716794bc4cc2bf2a5cfd0f1c59f32c28d79e00822e2d2eec5310344cd86f77f1527223ad569bd3c7d86d4bb57d34599c919e60a462f813fb2d0ebc757b1dcf6c213b1aae242f042e18ae67debd4862bd236214512aea62b4e992c639c3cdfbf3d2b8fde663e3a1136c733c7dba1260093239ea0d234b9c6d07dd5315190d23dae0c47958fa45bea2465380050f3258822fc9cc59eb53856355495b3efa1fd4810302e61d80c9a1743d0b1b0eac3111680a2d0fa2cce0b8bd5136186f3392590560d6d67e5e71a57a9b8887a98e7e36420485306d791ff7534f586e5ba8e2ceb010d

In [18]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[11768814371718170976, 435173728250646979, 519717007263840094, 1741290966923863957], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [9740814119635710701, 8723924064432029923, 17927155970413989335, 152971583043225146]]]
proof boolean:  1.0
proof result 1 : 50.86328125
verified
